In [3]:
import warnings
from xml.etree.ElementInclude import include
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import pandas as pd
from functools import reduce
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from statistics import mean
import numpy as np
import statsmodels.api as sm


import matplotlib
plt.style.use('ggplot')

matplotlib.rcParams['figure.figsize'] = (12,8)

pd.options.mode.chained_assignment = None

import seaborn as sns



In the current dataset you’re expected to track the user’s engagement using the following engagement metrics: 
sessions frequency 
the duration of the session 
the sessions total traffic (download and upload (bytes))???
How do we solve this problem?
- we recognise a user as MSISDN/NUMBER
- group the users based on the data used on the apps,
- determine what parameters can help portray engagement


In [4]:
df = pd.read_csv("../data/telcom.csv")

In [7]:
df['social_media_data'] = df['Social Media DL (Bytes)'] + df['Social Media UL (Bytes)']
df['google_data'] = df['Google DL (Bytes)'] + df['Google UL (Bytes)'] 
df['email_data'] = df['Email DL (Bytes)'] + df['Email UL (Bytes)'] 
df['youtube_data'] = df['Youtube DL (Bytes)'] + df['Youtube UL (Bytes)']
df['netflix_data'] = df['Netflix DL (Bytes)'] + df['Netflix UL (Bytes)']
df['gaming_data'] = df['Gaming DL (Bytes)'] + df['Gaming UL (Bytes)'] 
df['other_data'] = df['Other DL (Bytes)'] + df['Other UL (Bytes)']
df['total_data'] = df['Total UL (Bytes)'] + df['Total DL (Bytes)']

In [8]:
df.sc

,Bearer Id,Start,Start ms,End,End ms,Dur. (ms),IMSI,MSISDN/Number,IMEI,Last Location Name,...,Total UL (Bytes),Total DL (Bytes),social_media_data,google_data,email_data,youtube_data,netflix_data,gaming_data,other_data,total_data
0,1.311450e+19,4/4/19 12:01,770.0,4/25/19 14:35,662.0,1823652.0,2.082014e+14,3.366496e+10,3.552121e+13,9.16457E+15,...,36749741.0,308879636.0,1570185.0,2905912.0,3701304.0,18355943.0,17855187.0,292426453.0,180558843.0,345629377.0
1,1.311450e+19,4/9/19 13:04,235.0,4/25/19 8:15,606.0,1365104.0,2.082019e+14,3.368185e+10,3.579401e+13,L77566A,...,53800391.0,653384965.0,1933278.0,4414096.0,937385.0,39359124.0,35565545.0,609920783.0,541959383.0,707185356.0
2,1.311450e+19,4/9/19 17:42,1.0,4/25/19 11:58,652.0,1361762.0,2.082003e+14,3.376063e+10,3.528151e+13,D42335A,...,27883638.0,279807335.0,1726277.0,10229119.0,3363124.0,34425237.0,23751202.0,229980251.0,414908351.0,307690973.0
3,1.311450e+19,4/10/19 0:31,486.0,4/25/19 7:36,171.0,1321509.0,2.082014e+14,3.375034e+10,3.535661e+13,T21824A,...,43324218.0,846028530.0,657493.0,11811761.0,2070983.0,36534765.0,15092588.0,810387875.0,761837216.0,889352748.0
4,1.311450e+19,4/12/19 20:10,565.0,4/25/19 10:40,954.0,1089009.0,2.082014e+14,3.369980e+10,3.540701e+13,D88865A,...,38542814.0,569138589.0,912788.0,7748843.0,2110349.0,34222253.0,17539799.0,531237049.0,564619822.0,607681403.0
